# Preprocessing 

## 1) Import des données 

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv("../6.Data/kaggle_b2_fraud_train_v3.csv")

## 1) Valeurs null

### 1) On remplace par la médiane 

In [11]:
cols_median = [
    "annual_income_eur",
    "credit_score",
    "avg_amount_30d_eur",
    "max_amount_30d_eur"
]

df[cols_median] = df[cols_median].fillna(
    df[cols_median].median()
)

### 2) On remplace par la moyenne

In [12]:
cols_mean = ["device_trust_z", "ip_risk_z"]

df[cols_mean] = df[cols_mean].fillna(df[cols_mean].mean())


### 3) On change les valeurs en booléens et on remplace par 0

In [13]:
df["secondary_email"] = df["secondary_email"].notnull().astype(int)
df["legacy_partner_score"] = df["legacy_partner_score"].fillna(0)


### 4) On supprimes les colonnes pas utiles

In [14]:
cols_to_drop = ["occupation", "merchant_category", "customer_note", "last_ticket_subject",'region']
df = df.drop(columns=cols_to_drop)
